## Below you will see code which reads shapefile, DEM and climate information to generate a 2D array of per-pixel mass balance at a single time level.

You may use this code in task 2 to automate a procedure to calculate total mass balance for the entire climate time series. As you think about how to generate this time series using a for-loop, you should consider which commands below would need to be re-run for each time level, and which can be run just once for the entire process.

In [ ]:
# needed to read the elevation raster
import rasterio as rio

import numpy as np

# needed to read the climate data
import xarray as xr
import geopandas as gpd
import pandas as pd

# needed to generate a mask of pixels within the glacier boundary
from rasterio.mask import mask

In [ ]:
rgi = 'RGI60-16.01242'

# read in the DEM as an array named "z"
geotiffreader = rio.open('dems/dem_' + rgi + '.tif')
z = geotiffreader.read(1)

# use the geotiffreader.transform array to find cell area in m^2
cell_area = np.abs(geotiffreader.transform[0]*geotiffreader.transform[4])

In [ ]:
# read the RGI shapefile and set gdf to a new dataframe containing only 1 row
gdf = gpd.read_file('16_rgi60_LowLatitudes.shp')
ind = gdf.RGIId.str.fullmatch(rgi)
gdf = gdf[ind]

In [ ]:
# transform glacier outline from lat-lon to the UTM coordinate system of the DEM
gdf = gdf.to_crs(geotiffreader.crs)

In [ ]:
# generate a mask -- an array which contains elevation in pixels which are INSIDE the glacier,
# but are set to a "no data" value of -9999 outside
mask_glacier, _ = mask(geotiffreader, gdf.geometry, invert=False, nodata=-9999)

# mask_glacier has size (1,ny,nx) where (ny,nx) is the shape of the DEM -- ie a meaningless
# third dimension.
# this command gets rid of this "singleton" third dimension so it can be used
# in operations below.
mask_glacier = np.squeeze(mask_glacier)

In [ ]:
# read the climate data and extract temp and precip
ds = xr.open_dataset('clim/clim_rcp85_' + rgi + '.nc')
temperature = ds.temp.values
prcp = ds.prcp.values

In [ ]:
# extract data from first time level
temp_init = temperature[0]
prcp_init = prcp[0]

In [ ]:
# define parameters -- THESE ARE NOT THE ONES YOU ARE INSTRUCTED TO USE!!!
mu = 20;
lam = .006;
Tsolid = -2;
# define parameters -- THESE ARE NOT THE ONES YOU ARE INSTRUCTED TO USE!!!

# get temperature in every pixel using the temperature at the reference height, the elevation,
# and the lapse rate
temp_pixel = temp_init - (z-ds.ref_hgt)*lam

# get melt in each pixel (a negative number) according to degree day factor
smb_pixel = -mu * temp_pixel

# set melt to zero in pixels where temperature is negative or we are outside of the glacier
smb_pixel[(mask_glacier==-9999)|(temp_pixel<0)] = 0.0

# add the precipitation to pixels inside the glacier, only where temperature
# is below the threshold for snow
# (the backslash is to continue an expression on the next line)
smb_pixel[(mask_glacier>-9999) & (temp_pixel<Tsolid)] = \
 smb_pixel[(mask_glacier>-9999) & (temp_pixel<Tsolid)] + prcp_init

# smb_pixel is now a 2D array of mass balance per pixel